<a href="https://colab.research.google.com/github/jbm4/FOMCMinutesAnalysis/blob/main/FOMCMinutesRNNModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import pickle
import spacy
import re
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
merged_df=pd.read_csv('/content/drive/MyDrive/thinkful/Capstone 4/merged_df.csv')

In [ ]:
unrate=pd.read_csv('/content/drive/MyDrive/thinkful/Capstone 4/UNRATE.csv')
cpiaucsl=pd.read_csv('/content/drive/MyDrive/thinkful/Capstone 4/CPIAUCSL.csv')

In [ ]:
unrate.DATE=pd.to_datetime(unrate['DATE'])
cpiaucsl.DATE=pd.to_datetime(cpiaucsl.DATE)

In [ ]:
datetimes=['datetime_min', 'datetime_fed', 'next_month', 'DATE']
for dt in datetimes:
  merged_df[dt]=pd.to_datetime(merged_df[dt])

In [ ]:
merged_df=merged_df.merge(unrate, how='inner', on='DATE')
merged_df=merged_df.merge(cpiaucsl, how='inner', on='DATE')

In [ ]:
merged_df.head(1)

,date,filename,raw_text,preprocessed,datetime_min,next_month,DATE,FEDFUNDS,datetime_fed,change,change_cat,change_binary,change_1,change_binary_1,UNRATE,CPIAUCSL
0,19560417,fomchistmin19560417.txt,A meeting of the Federal Open Market Committee...,meet feder open market committe held offic boa...,1956-04-17,1956-05-01,1956-05-01,2.75,1956-05-01,0.13,1,1,-0.05,0,4.3,27.03


## Doc2Vec

In [ ]:
#encoding with Doc2Vec

documents=[TaggedDocument(doc, [i]) for i, doc in enumerate(merged_df.raw_text)]
model=Doc2Vec(documents)

In [ ]:
doc2vec=pd.DataFrame([[document]+list(model[document]) for document in range(len(documents))]).drop(0, axis=1)

In [ ]:
doc2vec.shape

(822, 100)

In [ ]:
doc2vec[['change_binary','datetime_min', 'datetime_fed']]=merged_df[['change_binary','datetime_min', 'datetime_fed']]

In [ ]:
doc2vec_1993=doc2vec[doc2vec['datetime_min'] > '1993-01-01']

In [ ]:
#random forest classifier model
x_train1993, x_test1993, y_train1993, y_test1993 = train_test_split(doc2vec_1993.drop(columns=['change_binary','datetime_min', 'datetime_fed']), doc2vec_1993['change_binary'], test_size=0.3)

rfc_model = RandomForestClassifier(n_estimators=75, min_samples_leaf=4, max_depth=12)

rfc_model.fit(x_train1993, y_train1993)
rfc_model_predictions = rfc_model.predict(x_test1993)
print(classification_report(y_test1993, rfc_model_predictions))

In [ ]:
rfc_model_predictions = rfc_model.predict(x_test1993)
print(classification_report(y_test1993, rfc_model_predictions))

              precision    recall  f1-score   support

           0       0.78      0.93      0.85        60
           1       0.33      0.11      0.17        18

    accuracy                           0.74        78
   macro avg       0.56      0.52      0.51        78
weighted avg       0.68      0.74      0.69        78



In [ ]:
doc2vec_1993.change_binary.value_counts()

0    184
1     74
Name: change_binary, dtype: int64

## Bert

In [ ]:
bert_df=merged_df[['filename', 'raw_text', 'datetime_min', 'datetime_fed', 'change', 'change_cat', 'change_1']]

In [ ]:
def text_cleaner(text):
  # Visual inspection identifies a form of punctuation that spaCy doesn't
  # recognize: the double dash --. Better get rid of it now!
  # text = re.sub(r'--',' ',text)
  # text = re.sub("[\[].*?[\]]", "", text)
  # text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
  # text = ' '.join(text.split())
  text= re.sub("Mr.", 'Mr', text)
  text= re.sub("Mrs.", 'Mrs', text)

  return text

nlp = spacy.load('en')
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated

#think about only keeping the sentences if it has a verb
def sentences(text):
  doc = nlp(text)
  sentences = [sent.string.strip() for sent in doc.sents]
  return sentences
def remove_segments(sentence_list):
  return [s for s in sentence_list if len(s.split())>2 ]

In [ ]:
bert_df['cleaned']=bert_df['raw_text'].apply(lambda x: sentences(text_cleaner(x)))
bert_df['cleaned']=bert_df['cleaned'].apply(lambda x: remove_segments(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#pickling for later
import pickle

#bert_df.to_pickle('/content/drive/MyDrive/thinkful/Capstone 4/merged_df.pkl')
bert_df=pickle.load(open('/content/drive/MyDrive/thinkful/Capstone 4/merged_df.pkl', 'rb'))


In [ ]:
#only working with 1993 and onward info for now
bert1993_df=bert_df[bert_df['datetime_min']>'1993-01-01']

#bert1993e_df=bert1993_df.explode('cleaned')

In [ ]:
bert_df.head(2)

,filename,raw_text,datetime_min,datetime_fed,change,change_cat,change_1,cleaned
0,fomchistmin19560417.txt,A meeting of the Federal Open Market Committee...,1956-04-17,1956-05-01,0.13,1,-0.05,[A meeting of the Federal Open Market Committe...
1,fomcminutes20090603.txt,Minutes of the Federal Open Market Committee \...,2009-06-03,2009-07-01,-0.05,0,-0.05,[Minutes of the Federal Open Market Committee ...


**Creating the model to embed the sentences**

In [ ]:
import pandas as pd
import numpy as np
import torch
!pip install transformers
import transformers as ppb
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

def embed_document(sentences_list):
  sentences_df=pd.DataFrame(sentences_list)
  tokenized = sentences_df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
      max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  attention_mask = np.where(padded != 0, 1, 0)

  input_ids = torch.tensor(padded) 
  attention_mask2 = torch.tensor(attention_mask)

  with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask2)
  features = last_hidden_states[0][:,0,:].numpy()

  return features
def CNN(features):
  

In [ ]:
bert1993_df['embedded']=bert1993_df['cleaned'].apply(lambda x: embed_document(x))

In [ ]:
bert1993_df.to_pickle('/content/drive/MyDrive/thinkful/Capstone 4/bert1993_df.pkl')


In [ ]:
tokenized = bert1993e_df['cleaned'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
import numpy as np

#saving padded and attention mask

#np.save('/content/drive/MyDrive/thinkful/Capstone 4/attention_mask.npy', attention_mask)
#np.save('/content/drive/MyDrive/thinkful/Capstone 4/padded.npy', padded)
attention_mask=np.load('/content/drive/MyDrive/thinkful/Capstone 4/attention_mask.npy')
padded=np.load('/content/drive/MyDrive/thinkful/Capstone 4/padded.npy')


In [ ]:
input_ids = torch.tensor(padded) 

In [ ]:
attention_mask2 = torch.tensor(attention_mask)

In [ ]:
import numpy as np
import torch
#np.save('/content/drive/MyDrive/thinkful/Capstone 4/attention_mask2.npy', attention_mask2)
#np.save('/content/drive/MyDrive/thinkful/Capstone 4/input_ids.npy', input_ids)
#attention_mask2=np.load('/content/drive/MyDrive/thinkful/Capstone 4/attention_mask2.npy')
#input_ids=np.load('/content/drive/MyDrive/thinkful/Capstone 4/input_ids.npy')


In [ ]:
with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask2)

#features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
attention_mask2

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [ ]:
#for use with bert

#TODO: representing the documents with a CNN
#tfidf_docs=pd.concat([tfidf_df, TFID1993_df[['filename']].reset_index(drop=True)], axis=1)
model=Sequential()

#first hidden layer
model.add(Conv2D(32,(3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#the above are part of the same layer
#stride is 1 by default


## RNN with TFID transformer:

In [ ]:
TFID1993_df=merged_df[merged_df['datetime_min']>'1993-01-01'][['filename',	'raw_text', 
                                                              'datetime_min',	'FEDFUNDS',	
                                                              'datetime_fed',	'change',	
                                                              'change_binary',	'change_1',	
                                                              'change_binary_1', 'UNRATE',	
                                                              'CPIAUCSL']]

In [ ]:
TFID1993_df.head(1)

,filename,raw_text,datetime_min,FEDFUNDS,datetime_fed,change,change_binary,change_1,change_binary_1,UNRATE,CPIAUCSL
1,fomcminutes20090603.txt,Minutes of the Federal Open Market Committee \...,2009-06-03,0.16,2009-07-01,-0.05,0,-0.05,0,9.5,214.726


In [ ]:
#could use this to filter out short sentences or sentences without verbs:
#sent_text = nltk.sent_tokenize(text)

def preprocess(doc):
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer('english')
    
    tokenized = word_tokenize(doc)

    cleaned = [stemmer.stem(lemmatizer.lemmatize(token.lower())) 
               for token in tokenized 
               if not token.lower() in stopwords.words('english') 
               if token.isalnum()] #should I make this also include numbers?
               

    untokenized = " ".join(cleaned)
    #preprocessed.append(untokenized)
        
    return untokenized

In [ ]:
TFID1993_df['lemmatized']=TFID1993_df['raw_text'].apply(lambda x: preprocess(x))

In [ ]:
#TFID1993_df.to_pickle('/content/drive/MyDrive/thinkful/Capstone 4/TFID1993_df.pkl')
#TFID1993_df=pickle.load(open('/content/drive/MyDrive/thinkful/Capstone 4/TFID1993_df.pkl', 'rb'))

TFID1993_df.head(1)

,filename,raw_text,datetime_min,FEDFUNDS,datetime_fed,change,change_binary,change_1,change_binary_1,UNRATE,CPIAUCSL,lemmatized
1,fomcminutes20090603.txt,Minutes of the Federal Open Market Committee \...,2009-06-03,0.16,2009-07-01,-0.05,0,-0.05,0,9.5,214.726,minut feder open market committe june 2009 joi...


In [ ]:
#getting the vector representation

vectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True)


# Applying the vectorizer
X = vectorizer.fit_transform(TFID1993_df["lemmatized"])
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
tfidf_df_1=tfidf_df.shift()
tfidf_merge = pd.concat([tfidf_df, tfidf_df_1], axis=1)

tfidf_merge = pd.concat([tfidf_merge, TFID1993_df.reset_index()], axis=1)
tfidf_merge.tail(1)

In [ ]:
tfidf_merge=tfidf_merge.dropna()

In [ ]:
from tensorflow.keras.layers import Input, Dense, Embedding,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential


In [ ]:
tfidf_df.shape

(258, 4836)

In [ ]:
#TODO: X and y have a different number of rows
X=pd.concat([tfidf_df, tfidf_df_1], axis=1).dropna()
y=tfidf_merge.reset_index(drop=True)['change_binary']
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=.25, random_state=1)

In [ ]:
n_timesteps, n_features = X_train.shape[0], X_train.shape[1]
n_outputs=1 #y_train[1]

model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='tanh'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100)               3909200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 3,919,401
Trainable params: 3,919,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
batch_size=80
epochs = 50

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=1) #batch_size=batch_size

ValueError: ignored

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

vocab_size = len(set(yelp.normalize.apply(lambda x: x.strip().split(' ')).sum())) # np.sum(yelp.text_norm.str.len())
yelp['encoded'] = yelp.text_norm.apply(one_hot, args=[vocab_size])

from tensorflow.keras.preprocessing.sequence import pad_sequences

independent_vars = pad_sequences(yelp['encoded'].values)
independent_vars

from sklearn.model_selection import train_test_split

target = yelp.sentiment
X_train, X_test, y_train, y_test = train_test_split(independent_vars, target, test_size=0.2, random_state=21)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

max_words = np.max(X_train)+1
max_len = X_train.shape[1]

model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_len))
model.add(SimpleRNN(32, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

epochs = 50
batch_size = 128

model.fit(X_train, y_train, validation_data=(X_test, y_test),
          batch_size=batch_size, epochs=epochs)

model.evaluate(X_test, y_test)